In [1]:
import pandas as pd

# read the CSV file into a DataFrame
data = pd.read_csv("./datasets/2_5_mph.csv")

# get a list of all column names
column_names = list(data.columns)

# initialize a counter for the new column names
count = 1

# iterate through the column names
for i in range(len(column_names)):
    if column_names[i] == "X [s]":
        if count == 1:
            new_name = "X [s].0"
        else:
            new_name = "X [s]." + str(count)
        data = data.rename(columns={column_names[i]: new_name})
        count += 1


# save the modified DataFrame to a new CSV fi
data.to_csv('output.csv', index=False)

In [2]:
import pandas as pd
from scipy import interpolate

from scipy.interpolate import interp1d

def resample_y1(x1, y1, x2):
    interp_func = interp1d(x1, y1, kind='linear')
    return interp_func(x2)

# read the CSV file and select the columns
data = pd.read_csv("./output.csv", usecols=["X [s].0", "L_Quad: EMG 1 [Volts]","X [s].1", "L_Quad: Acc 1.X [g]"])

# extract the data from the data frame
x1 = data["X [s].0"]
y1 = data["L_Quad: EMG 1 [Volts]"]
x2 = data["X [s].1"]
y2 = data["L_Quad: Acc 1.X [g]"]

# Resample y1 to match the sampling rate in x2
new_y1 = resample_y1(x1, y1, x2)

# Combine the data into a single table
data_combined = pd.DataFrame({'X [s]': x2, 'L_Quad: EMG 1 [Volts]': new_y1, 'L_Quad: Acc 1.X [g]': y2})

In [3]:
data_combined.head()

,X [s],L_Quad: EMG 1 [Volts],L_Quad: Acc 1.X [g]
0,0.00000,0.0,0.0
1,0.00675,0.0,0.0
2,0.01350,0.0,0.0
3,0.02025,0.0,0.0
4,0.02700,0.0,0.0


In [4]:
#data_combined.to_csv('output_resample.csv', index=False)

In [5]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

def resample_y(x1, y1, x2):
    interp_func = interp1d(x1, y1, kind='linear')
    return interp_func(x2)

# read the CSV file and select all columns
data = pd.read_csv("./output.csv")

# extract the time column and all EMG and Acc columns
time_col = data["X [s].0"]
emg_cols = [col for col in data.columns if "EMG" in col]
acc_cols = [col for col in data.columns if "Acc" in col]

# find the minimum and maximum times in the time column
min_time = time_col.min()
max_time = time_col.max()

# create a new time column with a uniform sampling rate
sample_rate = 100 # change this to the desired sampling rate
new_max_time = max_time - 1/sample_rate
new_time = pd.Series(data=np.arange(min_time, new_max_time+1/sample_rate, 1/sample_rate), name="X [s]")

# resample each EMG column to match the sampling rate in the new time column
emg_resampled = pd.DataFrame()
for col in emg_cols:
    new_y = resample_y(data["X [s].0"], data[col], new_time)
    emg_resampled[col] = new_y

# resample each Acc column to match the sampling rate in the new time column
acc_resampled = pd.DataFrame()
for col in acc_cols:
    new_y = resample_y(data["X [s].1"], data[col], new_time)
    acc_resampled[col] = new_y

# Combine the new time column, resampled EMG columns, and resampled Acc columns into a single table
data_combined = pd.concat([new_time, emg_resampled, acc_resampled], axis=1)

# Save the combined data to a CSV file
data_combined.to_csv("combined_output.csv", index=False)


In [6]:
# import pandas as pd
# import numpy as np
# from scipy.interpolate import interp1d

# def resample_y(x1, y1, x2):
#     interp_func = interp1d(x1, y1, kind='linear')
#     return interp_func(x2)

# # read the CSV file and select all columns
# data = pd.read_csv("./output.csv")

# # extract the time column and all EMG and Acc columns
# time_col = data.iloc[:,0]
# emg_cols = [col for col in data.columns if "EMG" in col]
# acc_cols = [col for col in data.columns if "Acc" in col]

# # find the sampling rate of the first EMG column
# first_emg_col = emg_cols[0]
# sample_rate = int(1/(time_col.iloc[1] - time_col.iloc[0]))

# # resample the Acc columns to match the sampling rate of the EMG columns
# acc_resampled = pd.DataFrame()
# for col in acc_cols:
#     new_y = resample_y(time_col, data[col], time_col[::int(sample_rate/500)])
#     acc_resampled[col] = new_y

# # Combine the original time column, EMG columns, and resampled Acc columns into a single table
# data_combined = pd.concat([time_col, data[emg_cols], acc_resampled], axis=1)

# # Save the combined data to a CSV file
# data_combined.to_csv("combined_output.csv", index=False)
